# Рекомендация тарифов

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

**План:**
1. Загрузить и ознакомиться с данными.
2. Подготовить данные к обучению модели.
3. Провести тест,и проверить адекватность модели.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.dummy import DummyClassifier

<div class="alert alert-info" role="alert">
Посмотрим на данные
</div>

In [6]:
df = pd.read_csv('users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<div class="alert alert-info" role="alert">
Проверим дубликаты
</div>

In [9]:
df.duplicated().sum()

0

<div class="alert alert-info" role="alert">
С данными все в порядке можно переходить исследованию моделей.
</div>

## Разбивка данных

<div class="alert alert-info" role="alert">
Дважды разбиваем данные чтобы получить 60% на обучение и по 20% для валидации и теста.
</div>

In [10]:
X = df.drop(['is_ultra'],axis=1)
y = df['is_ultra']

In [11]:
X_train,X_time_test,y_train,y_time_test = train_test_split(X,y,random_state = 12345,test_size = 0.4)
X_valid,X_test,y_valid,y_test = train_test_split(X_time_test,y_time_test,random_state = 12345,test_size = 0.5)

## Исследование модели

<div class="alert alert-info" role="alert">
Соберем лес классификаций.
</div>

In [12]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators = est)
    model.fit(X_train,y_train)
    result = model.score(X_valid,y_valid)
    if result > best_result:
        best_model = model
        best_result = result
display(best_model)
display(best_result)

RandomForestClassifier(n_estimators=10, random_state=12345)

0.7853810264385692

<div class="alert alert-info" role="alert">
0.78 нам подходит,но поищем модели получше.
</div>

<div class="alert alert-info" role="alert">
Пробуем модель логистической регрессии
</div>

In [13]:
model = LogisticRegression(random_state=12345,solver='lbfgs',max_iter=1000)
model.fit(X_train,y_train)
model.predict(X_valid)
accuracy = model.score(X_train, y_train)
display(accuracy)

0.7131742738589212

<div class="alert alert-info" role="alert">
Результат 0.71 хуже предыдущего.
</div>

<div class="alert alert-info" role="alert">
Лучшее accuracy вышло с леса классификаций,для теста будем использовать эту модель.
</div>

## Проверка модели на тестовой выборке

In [14]:
model = RandomForestClassifier(random_state=12345, n_estimators = 10)
model.fit(X_test,y_test)
test = model.score(X_test,y_test)
print(test)

0.9782270606531882


<div class="alert alert-info" role="alert">
На тесте получился отличный результат 0.98.
</div>

## Проверка модели на адекватность

In [15]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_train,y_train)
dummy_clf.predict(X)
dummy_clf.score(X_test,y_test)

0.6842923794712286

<div class="alert alert-info" role="alert">
Полученное значение модели на порядок выше чем на Dummy,модель прошла проверку
</div>

# Вывод

Загрузили и проверили данные.

Разделил данные на выборки для подачи в модель.

Обучили RandomForestClassifier. На тесте получен хороший результат accuracy: 0.97.
    
Сравнили с константной моделью DummyClassifier.Константная модель дает результат accuracy: 0.68,что говорит о том что наша модель проходит проверку на адекватность